In [1]:
from keys import ameritrade # store API key externally
import requests # library for making HTTP requests to API
import pandas as pd

In [ ]:
ameritrade

In [ ]:
# Read the complete list of companies into a dataframe
df = pd.read_excel('Complete_List.xlsx')
df

In [ ]:
# df[df.Symbol.str.contains(r'[@#&$%+-/*]')]

# Returns a match for any character except 0 to 9, a to z and A to Z
# df[df.Symbol.str.contains(r'[^0-9a-zA-Z]')]

# In addition to above do not include those with a '-'
# df[df.Symbol.str.contains(r'[^0-9a-zA-Z\-]')]

In [ ]:
# All symbols that contain '^', e.g.: DFNS^', cause an error with the API
# df[df['Symbol'].str.contains('\^')]

# Remove all that fulfill the condition
df['Symbol'] = df[df['Symbol'].str.contains('\^') == False]

In [ ]:
# Only use the 'Symbol' column and convert the series into a list
symbols = df['Symbol'].values.tolist()
symbols[:5]

In [ ]:
# url to make requests to
url = 'https://api.tdameritrade.com/v1/instruments'
# symbols[0] = 'CUE^'

In [ ]:
# # Check if any symbol is in the wrong format for the API
# index = 0

# # while index < 2:
# while index < len(symbols):
#     ticker = symbols[index]
# #     print(ticker)
#     print(index, ticker)

#     payload = {
#         'apikey': ameritrade,
#         'symbol': ticker,
#         'projection': 'symbol-search'
#     }
    
#     try:
#         results = requests.get(url, params=payload) # get request for tickers to API
# #         print(results.status_code)
#         if results.status_code == 400:
#             print('wrong format with', ticker, 'index', index)
# #             symbols.pop(index) # remove symbol from the list
# #             df.drop(df.index[index], inplace=True)
#     except:
#         print('data could not be retrieved from the API')
    
#     index += 1

In [ ]:
# # Save cleaned DataFrame to csv
# df.to_csv('Edited_List.csv', index=False)

In [ ]:
# # Only use the 'Symbol' column and convert the series into a list
# symbols = df['Symbol'].values.tolist()
# symbols[:5]

In [ ]:
for i in symbols[:10]:
    print(i)

In [2]:
# Retrieve SP500 stock list from datahub API (https://datahub.io/core/s-and-p-500-companies#pandas)

import datapackage

data_url = 'https://datahub.io/core/s-and-p-500-companies/datapackage.json'

# to load Data Package into storage
package = datapackage.Package(data_url)

# to load only tabular data
resources = package.resources
for resource in resources:
    if resource.tabular:
        data = pd.read_csv(resource.descriptor['path'])
        print (data)

    Symbol                 Name                  Sector
0      MMM           3M Company             Industrials
1      AOS      A.O. Smith Corp             Industrials
2      ABT  Abbott Laboratories             Health Care
3     ABBV          AbbVie Inc.             Health Care
4     ABMD              Abiomed             Health Care
..     ...                  ...                     ...
500    YUM      Yum! Brands Inc  Consumer Discretionary
501   ZBRA   Zebra Technologies  Information Technology
502    ZBH        Zimmer Biomet             Health Care
503   ZION        Zions Bancorp              Financials
504    ZTS               Zoetis             Health Care

[505 rows x 3 columns]
    Symbol                 Name                  Sector
0      MMM           3M Company             Industrials
1      AOS      A.O. Smith Corp             Industrials
2      ABT  Abbott Laboratories             Health Care
3     ABBV          AbbVie Inc.             Health Care
4     ABMD              

In [ ]:
# url to make requests to
url = 'https://api.tdameritrade.com/v1/instruments'

In [ ]:
payload = {
    'apikey': ameritrade,
    'symbol': 'GOOGL',
    'projection': 'fundamental'
}

In [ ]:
# make a get request
results = requests.get(url, params=payload)
results

In [ ]:
# returned attributes fo the get request
dir(results)

In [ ]:
# retrieve json object attribute
results.json()

In [ ]:
# # loop through first 10 symbols and retrieve pegRation
# for i in symbols[:10]:
#     print(i)
#     payload = {
#         'apikey': ameritrade,
#         'symbol': i,
#         'projection': 'fundamental'
#     }
#     dataType = payload['projection']
    
#     try:
#         results = requests.get(url, params=payload)
# #         print(results)
#         data = results.json()
# #         print(data)
# #         print(data[i])
#         print(data[i][dataType]['pegRatio']) #return the pegRatio
#     except:
#         print('data could not be retrieved from the API')

In [ ]:
import time # required for naming the files based on time
import pickle as pkl # save and load file
import os # for removing the files

In [ ]:
# # Name files based on time, replace spaces and semicolon
# f_name = 'data/' + time.asctime() + '.pkl'
# f_name = f_name.replace(' ', '_').replace(':', '_')

In [ ]:
# Only a certain number of stocks can be retrieved at a time
stock_range = 500

start = 0
end = start + stock_range

files = []

while start < len(symbols):
    tickers = symbols[start:end]
#     print(tickers)

    payload = {
        'apikey': ameritrade,
        'symbol': tickers,
        'projection': 'fundamental'
    }
    
    try:
        results = requests.get(url, params=payload) # get request for tickers to API
        data = results.json() # retrieve json object
        # save file named after current time to data folder
        f_name = 'data/' + time.asctime() + '.pkl'
        f_name = f_name.replace(' ', '_').replace(':', '_')    
        files.append(f_name)
        
        # use with statement, this makes sure that file is closed at the end of the block
        with open(f_name, 'wb') as file:
            pkl.dump(data, file)
    except:
        print('data could not be retrieved from the API')
    
    start = end
    end += stock_range
    time.sleep(1) # wait for 1 sec, required so that each batch will be 1 sec apart, which is required so that all files have different names
    
#     print(start, end)

# print(len(symbols))

In [ ]:
# directory = {
#     'AGLE': {
#             'fundamental': {
#                      'symbol': 'AGLE', 
#                      'high52': 11.38, 
#                      'low52': 3.5, 
#                      'dividendAmount': 0.0, 
#                      'dividendYield': 0.0, 
#                      'dividendDate': ' ', 
#                      'peRatio': 0.0
#             }
#     }
# }
# directory

In [ ]:
# list(directory)

In [ ]:
# Retrieve data from stored files
tickerData = []
# for file in files[:1]:
# loop through all files
for file_number, file in enumerate(files):
    with open(file, 'rb') as f:
        info = pkl.load(f)
    
    try:
        tickers = list(info) # retrieve only the tickers from the dictionary (key of outermost dictionaries)
    #     print(info)
#         print(tickers, file_number)
        points = ['symbol', 'netProfitMarginMRQ', 'peRatio', 'pegRatio', 'high52']
        for ticker in tickers:
            tick = []
            for point in points:
    #             print(info[ticker]['fundamental'][point])
                tick.append(info[ticker]['fundamental'][point]) # store each fundamental data
            tickerData.append(tick) # store the fundamental data for each tick in tickerData
    #     os.remove(file) # delete file after storing the data
    
    except:
        print('Error with file_number:', file_number, info)
    

In [ ]:
stockData = pd.DataFrame(tickerData, columns=points)
stockData

In [ ]:
# Return all stocks with a pegRatio greater 1
stockData_peg = stockData[stockData['pegRatio'] > 1]
stockData_peg